<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%%bash
curl -O https://covid.ourworldindata.org/data/owid-covid-data.csv
mv owid-covid-data.csv data/owid-covid-data.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2372k  100 2372k    0     0   320k      0  0:00:07  0:00:07 --:--:--  339k


In [2]:
import sys
sys.path.append("..")
sys.path.append("../src")

import yaml
config = yaml.load(open('../src/configs/config.yaml', 'r'))

/Users/fernandascovino/Projects/simulacovid/venvanalysis/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [3]:
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

In [4]:
# import plotly.io as pio

In [5]:
df = pd.read_csv('data/owid-covid-data.csv')
df

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,tests_units,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,0.0,NaN,NaN,NaN,NaN,NaN,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,0.0,NaN,NaN,NaN,NaN,NaN,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17008,NaN,International,2020-02-28,705,0,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17009,NaN,International,2020-02-29,705,0,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17010,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17011,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_rolling_amount(grp):
    return grp.rolling(5, min_periods=1, on='date')['new_deaths'].mean()

df_deaths = df[~df['new_deaths'].isnull()][['iso_code', 'date','total_deaths', 'new_deaths']]\
                                                .groupby(['iso_code', 'date','total_deaths'])['new_deaths']\
                                                .sum()\
                                                .reset_index()

df_deaths['rolling_deaths_new'] = df_deaths.groupby('iso_code', 
                                                    as_index=False, 
                                                    group_keys=False)\
                                           .apply(get_rolling_amount)

In [7]:
df_deaths = df_deaths[df_deaths['iso_code']!='OWID_WRL']

In [8]:
def _df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

In [10]:
def plot_deaths_heatmap2(t, place_type, min_deaths, title,save_img=False):
    
    df_heatmap = t.reset_index()\
                          .pivot(index=place_type, 
                                  columns='date', 
                                  values='rolling_deaths_new')\
                           .fillna(0)\
                           .apply(lambda x: x/x.max(), axis=1)\
                           .dropna(how='all')

    # remove days with all states zero
    df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
    print(len(df_heatmap))
    states_total_deaths = [t[t['iso_code'] ==x]['total_deaths'].max() for x in df_heatmap.index]
    idy = np.argsort(states_total_deaths)
    idx = np.array(states_total_deaths,dtype=np.int32)[idy] > min_deaths
    
    

    trace1 = go.Heatmap(_df_to_plotly(df_heatmap.iloc[idy,:].loc[idx,:]), 
                                        colorscale='temps',showscale=False)


    trace2 = go.Bar(x=np.array(states_total_deaths)[idy][idx],y=df_heatmap.iloc[idy,:].loc[idx,:].index,
                    xaxis="x2",yaxis="y2",orientation='h')
        
    d = [trace1,trace2]
    layout = go.Layout(title=title,
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            domain=[0, 0.8]
        ),
        xaxis2=dict(
            domain=[0.85, 1]
        ),
        yaxis=dict(tickmode='linear'
        ),
        yaxis2=dict(tickmode='linear',anchor='x2'
        )
        )

    fig = go.Figure(data=d, layout=layout)
#     if save_img:
#         pio.write_image(fig,'Mortes_Mundo.jpeg',format='jpeg',width=1920, height=1080)
    fig.show()


In [11]:

plot_deaths_heatmap2(df_deaths,
                    place_type='iso_code',
                    min_deaths=1000,
                    title='Distribuição de novas mortes Mundo',
                    save_img=True)

175
